<a href="https://colab.research.google.com/github/zhenyavoronin97-collab/DS/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import pandas as pd
import requests
from typing import Optional, Dict, Any

class DataLoader:
    """Класс для загрузки данных из различных источников."""

    @staticmethod
    def load_csv(file_path: str, **kwargs) -> pd.DataFrame:
        try:
            df = pd.read_csv(file_path, **kwargs)
            print(f"CSV-файл '{file_path}' успешно загружен. Размер: {df.shape}")
            return df
        except Exception as e:
            print(f"Ошибка при загрузке CSV: {e}")
            raise

    @staticmethod
    def load_json(file_path: str, **kwargs) -> pd.DataFrame:
        try:
            df = pd.read_json(file_path, **kwargs)
            print(f"JSON-файл '{file_path}' успешно загружен. Размер: {df.shape}")
            return df
        except Exception as e:
            print(f"Ошибка при загрузке JSON: {e}")
            raise

    @staticmethod
    def load_api(url: str, params: Optional[Dict[str, Any]] = None) -> pd.DataFrame:
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            if isinstance(data, list):
                df = pd.DataFrame(data)
            elif isinstance(data, dict):
                df = pd.json_normalize(data)
            else:
                raise ValueError("Не удалось преобразовать ответ API в DataFrame")
            print(f"Данные из API '{url}' успешно загружены. Размер: {df.shape}")
            return df
        except Exception as e:
            print(f"Ошибка при загрузке из API: {e}")
            raise

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, List

class DataProcessor:
    """Класс для обработки DataFrame и визуализации."""

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.plots = []  # список созданных графиков

    # ---------- Пропуски ----------
    def missing_values_count(self) -> pd.Series:
        return self.df.isnull().sum()

    def missing_values_report(self) -> pd.DataFrame:
        missing_count = self.missing_values_count()
        missing_percent = (missing_count / len(self.df)) * 100
        report = pd.DataFrame({
            'Количество пропусков': missing_count,
            'Доля пропусков (%)': missing_percent
        }).sort_values('Доля пропусков (%)', ascending=False)
        print("Отчёт о пропущенных значениях:")
        print(report)
        return report

    def fill_missing(self, column: str, method: str = 'mean', **kwargs) -> None:
        if column not in self.df.columns:
            raise ValueError(f"Столбец '{column}' не найден")
        if method == 'mean':
            if not np.issubdtype(self.df[column].dtype, np.number):
                raise TypeError("Метод 'mean' применим только к числовым столбцам")
            fill_value = self.df[column].mean()
        elif method == 'median':
            if not np.issubdtype(self.df[column].dtype, np.number):
                raise TypeError("Метод 'median' применим только к числовым столбцам")
            fill_value = self.df[column].median()
        elif method == 'mode':
            mode_vals = self.df[column].mode(dropna=True)
            if len(mode_vals) == 0:
                raise ValueError(f"Нет моды для столбца '{column}'")
            fill_value = mode_vals[0]
        elif method == 'constant':
            if 'value' not in kwargs:
                raise ValueError("Для метода 'constant' необходимо указать параметр 'value'")
            fill_value = kwargs['value']
        else:
            raise ValueError("Метод должен быть 'mean', 'median', 'mode' или 'constant'")
        self.df[column].fillna(fill_value, inplace=True)
        print(f"Пропуски в столбце '{column}' заполнены методом '{method}' (значение: {fill_value})")

    # ---------- Основные методы визуализации ----------
    def add_histogram(self, column: str, bins: int = 30, **kwargs) -> None:
        if column not in self.df.columns:
            raise ValueError(f"Столбец '{column}' не найден")
        if not np.issubdtype(self.df[column].dtype, np.number):
            raise TypeError("Гистограмма строится только для числовых столбцов")
        fig, ax = plt.subplots(figsize=(10, 6))
        self.df[column].hist(bins=bins, ax=ax, **kwargs)
        ax.set_title(f'Распределение: {column}')
        ax.set_xlabel(column)
        ax.set_ylabel('Частота')
        self.plots.append(fig)
        plt.close(fig)
        print(f"Гистограмма для '{column}' добавлена")

    def add_lineplot(self, x: str, y: str, **kwargs) -> None:
        for col in [x, y]:
            if col not in self.df.columns:
                raise ValueError(f"Столбец '{col}' не найден")
        if not np.issubdtype(self.df[x].dtype, np.number) or not np.issubdtype(self.df[y].dtype, np.number):
            raise TypeError("Линейный график требует числовые столбцы")
        fig, ax = plt.subplots(figsize=(10, 6))
        sorted_df = self.df.sort_values(by=x)
        ax.plot(sorted_df[x], sorted_df[y], **kwargs)
        ax.set_title(f'{y} от {x}')
        ax.set_xlabel(x)
        ax.set_ylabel(y)
        self.plots.append(fig)
        plt.close(fig)
        print(f"Линейный график ({y} от {x}) добавлен")

    def add_scatter(self, x: str, y: str, hue: Optional[str] = None, **kwargs) -> None:
        for col in [x, y]:
            if col not in self.df.columns:
                raise ValueError(f"Столбец '{col}' не найден")
        if not np.issubdtype(self.df[x].dtype, np.number) or not np.issubdtype(self.df[y].dtype, np.number):
            raise TypeError("Диаграмма рассеяния требует числовые столбцы")
        fig, ax = plt.subplots(figsize=(10, 6))
        if hue is not None:
            if hue not in self.df.columns:
                raise ValueError(f"Столбец '{hue}' не найден")
            sns.scatterplot(data=self.df, x=x, y=y, hue=hue, ax=ax, **kwargs)
        else:
            ax.scatter(self.df[x], self.df[y], **kwargs)
        ax.set_title(f'{y} vs {x}')
        ax.set_xlabel(x)
        ax.set_ylabel(y)
        self.plots.append(fig)
        plt.close(fig)
        print(f"Scatter plot ({y} vs {x}) добавлен")

    # ---------- Дополнительные методы визуализации для Spotify ----------
    def add_barplot(self, x: str, y: str, title: str = "", **kwargs) -> None:
        """Столбчатая диаграмма (например, топ-10 исполнителей)"""
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.barplot(data=self.df, x=x, y=y, ax=ax, **kwargs)
        ax.set_title(title)
        ax.set_xlabel(x)
        ax.set_ylabel(y)
        plt.xticks(rotation=45, ha='right')
        self.plots.append(fig)
        plt.close(fig)
        print(f"Барплот '{title}' добавлен")

    def add_heatmap(self, columns: List[str], title: str = "Корреляция признаков") -> None:
        """Тепловая карта корреляций"""
        corr_matrix = self.df[columns].corr()
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=ax, fmt='.2f')
        ax.set_title(title)
        self.plots.append(fig)
        plt.close(fig)
        print("Тепловая карта добавлена")

    def add_boxplot(self, x: str, y: str, title: str = "") -> None:
        """Ящик с усами (например, распределение популярности по годам)"""
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.boxplot(data=self.df, x=x, y=y, ax=ax)
        ax.set_title(title)
        plt.xticks(rotation=45, ha='right')
        self.plots.append(fig)
        plt.close(fig)
        print(f"Boxplot '{title}' добавлен")

def add_heatmap(self, columns: list, title: str = "Корреляция признаков") -> None:
    """Тепловая карта корреляций для указанных столбцов."""
    corr_matrix = self.df[columns].corr()
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=ax, fmt='.2f')
    ax.set_title(title)
    self.plots.append(fig)
    plt.close(fig)
    print(f"Тепловая карта '{title}' добавлена")

    def remove_last_plot(self) -> None:
        if self.plots:
            self.plots.pop()
            print(f"График удалён. Осталось: {len(self.plots)}")
        else:
            print("Список графиков пуст")

    def show_all_plots(self) -> None:
        if not self.plots:
            print("Нет графиков для отображения")
            return
        for fig in self.plots:
            fig.show()
        plt.show()

    def check_data_types(self) -> pd.Series:
        return self.df.dtypes

    def summary_statistics(self) -> pd.DataFrame:
        return self.df.describe(include='all')


In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from data_loader import DataLoader
from data_processing import DataProcessor

# Настройка стиля графиков
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

loader = DataLoader()
df = loader.load_csv('spotify_2024.csv', encoding='utf-8')

print("\nПервые 5 строк:")
print(df.head())

print("\nИнформация о датасете:")
print(df.info())

# Переименуем столбцы для удобства (если нужно)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('%', '_percent')

print("Названия столбцов после нормализации:")
print(df.columns.tolist())

# Создаём процессор
proc = DataProcessor(df)

print("="*60)
print("АНАЛИЗ ПРОПУСКОВ")
print("="*60)
proc.missing_values_report()

print("="*60)
print("ЗАПОЛНЕНИЕ ПРОПУСКОВ")
print("="*60)

# Числовые столбцы
numeric_cols = proc.df.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    if proc.df[col].isnull().any():
        proc.fill_missing(col, method='mean')

# Категориальные столбцы
cat_cols = proc.df.select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    if proc.df[col].isnull().any():
        proc.fill_missing(col, method='mode')

print("\nПосле заполнения:")
print(proc.missing_values_count())

if 'popularity' in proc.df.columns:
    proc.add_histogram('popularity', bins=30, color='green', edgecolor='black', alpha=0.7)
    print("Гистограмма популярности добавлена")

if 'danceability_%' in proc.df.columns or 'danceability__' in proc.df.columns:
    dance_col = 'danceability_%' if 'danceability_%' in proc.df.columns else 'danceability__'
    proc.add_histogram(dance_col, bins=30, color='blue', edgecolor='black', alpha=0.7)
    print(f"Гистограмма танцевальности ({dance_col}) добавлена")

if 'energy_%' in proc.df.columns or 'energy__' in proc.df.columns:
    energy_col = 'energy_%' if 'energy_%' in proc.df.columns else 'energy__'
    proc.add_histogram(energy_col, bins=30, color='orange', edgecolor='black', alpha=0.7)
    print(f"Гистограмма энергии ({energy_col}) добавлена")

# Определяем переменные для столбцов
energy_col = None
dance_col = None

if 'energy_%' in proc.df.columns or 'energy__' in proc.df.columns:
    energy_col = 'energy_%' if 'energy_%' in proc.df.columns else 'energy__'

if 'danceability_%' in proc.df.columns or 'danceability__' in proc.df.columns:
    dance_col = 'danceability_%' if 'danceability_%' in proc.df.columns else 'danceability__'

# Теперь используем их, только если они определены
if energy_col and dance_col:
    # строим график

   if all(col in proc.df.columns for col in [energy_col, dance_col]):
    pass #
    temp_df = proc.df[[energy_col, dance_col]].dropna()
    if len(temp_df) > 0:
        temp_proc = DataProcessor(temp_df)
        temp_proc.add_scatter(x=energy_col, y=dance_col, alpha=0.5, color='purple')
        proc.plots.extend(temp_proc.plots)
        print(f"Scatter plot: {energy_col} vs {dance_col} добавлен")

if all(col in proc.df.columns for col in ['track_name', 'streams']):
    # Преобразуем streams в число (убираем запятые)
    proc.df['streams_clean'] = pd.to_numeric(proc.df['streams'].astype(str).str.replace(',', ''), errors='coerce')
    top_tracks = proc.df.nlargest(10, 'streams_clean')[['track_name', 'streams_clean']]

    fig, ax = plt.subplots(figsize=(12, 6))
    sns.barplot(data=top_tracks, x='streams_clean', y='track_name', palette='viridis', ax=ax)
    ax.set_title('Топ-10 самых популярных треков по количеству прослушиваний')
    ax.set_xlabel('Количество прослушиваний')
    ax.set_ylabel('Название трека')
    proc.plots.append(fig)
    plt.close(fig)
    print("Топ-10 треков добавлен")

if 'artist(s)_name' in proc.df.columns:
    top_artists = proc.df['artist(s)_name'].value_counts().head(10).reset_index()
    top_artists.columns = ['artist', 'count']

    fig, ax = plt.subplots(figsize=(12, 6))
    sns.barplot(data=top_artists, x='count', y='artist', palette='rocket', ax=ax)
    ax.set_title('Топ-10 исполнителей по количеству треков в датасете')
    ax.set_xlabel('Количество треков')
    ax.set_ylabel('Исполнитель')
    proc.plots.append(fig)
    plt.close(fig)
    print("Топ-10 исполнителей добавлен")

if 'released_year' in proc.df.columns:
    year_counts = proc.df['released_year'].value_counts().sort_index()

    fig, ax = plt.subplots(figsize=(14, 6))
    year_counts.plot(kind='bar', ax=ax, color='teal', alpha=0.7)
    ax.set_title('Количество треков по годам выпуска')
    ax.set_xlabel('Год')
    ax.set_ylabel('Количество треков')
    plt.xticks(rotation=45)
    proc.plots.append(fig)
    plt.close(fig)
    print("Распределение по годам добавлено")

# Выбираем числовые признаки, связанные с музыкой
music_features = []
for col in proc.df.columns:
    if any(x in col.lower() for x in ['danceability', 'energy', 'valence', 'acousticness',
                                       'instrumentalness', 'liveness', 'speechiness', 'bpm']):
        if col in proc.df.select_dtypes(include=[np.number]).columns:
            music_features.append(col)

if len(music_features) > 5:  # Если достаточно признаков
    proc.add_heatmap(music_features, title="Корреляция музыкальных характеристик")

if all(col in proc.df.columns for col in ['released_year', 'streams_clean']):
    # Возьмём только последние 10 лет для наглядности
    recent_df = proc.df[proc.df['released_year'] >= 2014].copy()
    if len(recent_df) > 0:
        fig, ax = plt.subplots(figsize=(14, 6))
        sns.boxplot(data=recent_df, x='released_year', y='streams_clean', ax=ax)
        ax.set_title('Распределение прослушиваний по годам (последние 10 лет)')
        ax.set_xlabel('Год выпуска')
        ax.set_ylabel('Количество прослушиваний')
        plt.xticks(rotation=45)
        proc.plots.append(fig)
        plt.close(fig)
        print("Boxplot по годам добавлен")

if all(col in proc.df.columns for col in ['explicit', 'streams_clean']):
    # Преобразуем explicit в категориальный, если нужно
    if proc.df['explicit'].dtype == 'bool' or proc.df['explicit'].dtype == 'object':
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.boxplot(data=proc.df, x='explicit', y='streams_clean', ax=ax)
        ax.set_title('Влияние explicit-контента на популярность')
        ax.set_xlabel('Explicit')
        ax.set_ylabel('Количество прослушиваний')
        proc.plots.append(fig)
        plt.close(fig)
        print("Анализ explicit-контента добавлен")

print(f"\nВсего создано графиков: {len(proc.plots)}")

# Удалим последний график для демонстрации (если хотите)
# proc.remove_last_plot()
# print(f"После удаления: {len(proc.plots)}")

print("\nОткрытие окон с графиками...")
proc.show_all_plots()

print("="*60)
print("ИТОГОВАЯ СТАТИСТИКА")
print("="*60)
print("\nТипы данных:")
print(proc.check_data_types())

print("\nСтатистическое описание:")
print(proc.summary_statistics())

# Ключевые инсайты
print("\n" + "="*60)
print("КЛЮЧЕВЫЕ ИНСАЙТЫ")
print("="*60)

if 'streams_clean' in proc.df.columns:
    print(f"Среднее количество прослушиваний: {proc.df['streams_clean'].mean():,.0f}")
    print(f"Медиана прослушиваний: {proc.df['streams_clean'].median():,.0f}")
    print(f"Максимум прослушиваний: {proc.df['streams_clean'].max():,.0f}")

if 'artist(s)_name' in proc.df.columns:
    top_artist = proc.df['artist(s)_name'].value_counts().index[0]
    print(f"Самый часто встречающийся исполнитель: {top_artist}")


CSV-файл 'spotify_2024.csv' успешно загружен. Размер: (953, 25)

Первые 5 строк:
                            track_name    artist(s)_name  artist_count  \
0  Seven (feat. Latto) (Explicit Ver.)  Latto, Jung Kook             2   
1                                 LALA       Myke Towers             1   
2                              vampire    Olivia Rodrigo             1   
3                         Cruel Summer      Taylor Swift             1   
4                       WHERE SHE GOES         Bad Bunny             1   

   released_year  released_month  released_day  in_spotify_playlists  \
0           2023               7            14                   553   
1           2023               3            23                  1474   
2           2023               6            30                  1397   
3           2019               8            23                  7858   
4           2023               5            18                  3133   

   in_spotify_charts    streams  in_apple

/content/data_processing.py:64: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


/tmp/ipython-input-1326838114.py:100: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=top_tracks, x='streams_clean', y='track_name', palette='viridis', ax=ax)
/tmp/ipython-input-1326838114.py:113: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. 

Топ-10 исполнителей добавлен
Распределение по годам добавлено


AttributeError: 'DataProcessor' object has no attribute 'add_heatmap'